In [19]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import numpy as np
from rouge import Rouge 

REFERENCE_PATH = "test/reference_summaries/"
PYTEASER_PATH = "test/system_summaries/pyteaser/"
GENSIM_PATH = "test/system_summaries/gensim/"

def rouge_e(system_summary, gold_summary):
    system_sentences = split_into_sentences(system_summary)
    gold_sentences = split_into_sentences(gold_summary)
    total_sys_words = 0
    for s_sent in system_sentences:
        s_words = word_tokenize(s_sent)
        total_sys_words+=len(s_words)
        ds = []
        for g_sent in gold_sentences:
            g_words = word_tokenize(g_sent)
            ed = editDistDP(s_words, g_words)
            ds.append(ed)
            
        print(ds, np.argmin(ds))
    
    pass

strs = """Ad sales boost Time Warner profit
TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.
However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues.
It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL's existing customers for high-speed broadband."""

strs1 = """TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn.Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues.Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins.It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters.Time Warner's fourth quarter profits were slightly better than analysts' expectations."""
rouge_e(strs, strs1)


[6, 26, 23, 21, 21, 23, 18, 16] 0
[20, 26, 26, 0, 23, 23, 22, 22] 3
[27, 28, 27, 27, 26, 27, 25, 28] 6


In [12]:
# A Dynamic Programming based Python program for edit 
# distance problem 
def editDistDP(list1, list2):
    m = len(list1)
    n = len(list2)
    # Create a table to store results of subproblems 
    dp = [[0 for x in range(n+1)] for x in range(m+1)] 
  
    # Fill d[][] in bottom up manner 
    for i in range(m+1): 
        for j in range(n+1): 
  
            # If first string is empty, only option is to 
            # insert all characters of second string 
            if i == 0: 
                dp[i][j] = j    # Min. operations = j 
  
            # If second string is empty, only option is to 
            # remove all characters of second string 
            elif j == 0: 
                dp[i][j] = i    # Min. operations = i 
  
            # If last characters are same, ignore last char 
            # and recur for remaining string 
            elif list1[i-1] == list2[j-1]: 
                dp[i][j] = dp[i-1][j-1] 
  
            # If last character are different, consider all 
            # possibilities and find minimum 
            else: 
                dp[i][j] = 1 + min(dp[i][j-1],        # Insert 
                                   dp[i-1][j],        # Remove 
                                   dp[i-1][j-1])    # Replace 
  
    return dp[m][n] 

st1 = ["haha", "it", "my","alakası yok" , "di"  , "mi"]
st2 = ["hello",  "ss", "fuck"]
print(editDistDP(st1, st2))

6


In [8]:
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])"
def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [52]:
import os

pyteaser_files = os.listdir(PYTEASER_PATH)
gensim_files = os.listdir(GENSIM_PATH)
gold_files = os.listdir(REFERENCE_PATH)
fout = open("results.csv", "w")
fout.write("DOCUMENT NAME, SYSTEM NAME, ROGUE-1 P, ROUGE-1 R, ROUGE-1 F, ROGUE-2 P, ROUGE-2 R, ROUGE-2 F, ROGUE-L P, ROUGE-L R, ROUGE-L F, ROUGE-E\n")
for p_file in pyteaser_files:
    
    f = open(PYTEASER_PATH + p_file , "r", encoding="latin-1")
    pyteaser_summ = f.read()
    
    docname = p_file.split("_")[0]
    print(docname)
    ref_file = docname+"_reference.txt"
    f = open(REFERENCE_PATH + ref_file, "r", encoding="latin-1")
    golden_summ = f.read()
    
    gensim_file = docname + "_gensim.txt"
    f = open(GENSIM_PATH + gensim_file, "r", encoding="latin-1")
    gensim_sum = f.read()

    rouge = Rouge()
    pyteaser_scores = rouge.get_scores(pyteaser_summ, golden_summ)
    gensim_scores = rouge.get_scores(gensim_sum, golden_summ)
    pyteaser_scores = [x for x in [list(x.values()) for x in list(pyteaser_scores[0].values())]]
    gensim_scores = [x for x in [list(x.values()) for x in list(gensim_scores[0].values())]]
    
    py_str = docname+ ","+ "pyteaser"+ ","+ ",".join(str(v) for v in pyteaser_scores[0])+ "," + ",".join(str(v) for v in pyteaser_scores[1])+ ","+ ",".join(str(v) for v in pyteaser_scores[2])+ "\n"
    gen_str = docname+ ","+ "gensim"+ ","+ ",".join(str(v) for v in gensim_scores[0])+ "," + ",".join(str(v) for v in gensim_scores[1])+ ","+ ",".join(str(v) for v in gensim_scores[2])+ "\n"
    fout.write(py_str)
    fout.write(gen_str)

fout.close()

entertainment256
sport174
tech086
entertainment224
business096
sport392
sport415
business132
sport357
entertainment059
tech032
sport266
entertainment073
sport315
tech261
tech290
politics117
politics113
business164
business326
sport212
politics355
business298
business321
entertainment179
business129
entertainment105
business383
politics131
entertainment075
business197
entertainment364
entertainment382
sport307
entertainment266
sport334
entertainment028
entertainment283
sport048
sport254
entertainment290
tech257
politics216
entertainment176
sport419
sport496
sport323
sport447
sport372
business406
tech228
tech035
sport386
politics065
tech279
sport042
tech372
sport231
business508
tech247
entertainment158
tech209
politics182
politics056
tech043
business128
tech027
entertainment346
tech147
business394
sport229
politics001
sport354
business223
politics364
tech378
sport227
entertainment115
business169
entertainment278
tech037
entertainment285
tech112
entertainment207
sport092
sport286
tech089


tech144
sport074
sport280
tech103
politics184
entertainment269
business500
tech076
sport190
entertainment268
business485
business147
entertainment043
entertainment056
business410
sport471
politics262
politics081
business276
tech242
entertainment319
sport285
politics280
entertainment232
sport103
entertainment037
sport224
business351
business009
sport281
sport061
business348
tech059
entertainment164
entertainment292
tech042
entertainment106
entertainment295
business248
tech260
entertainment048
tech383
politics140
sport163
entertainment046
entertainment155
tech074
sport088
politics141
business281
sport152
sport432
sport149
politics157
sport320
politics292
tech336
politics180
sport237
entertainment241
sport136
sport141
business346
politics192
sport062
sport480
entertainment111
sport098
sport290
sport276
business243
tech332
entertainment231
sport267
politics172
entertainment189
politics160
business404
politics080
entertainment174
sport448
business439
business414
tech366
business494
politics

sport195
politics109
business027
sport086
sport072
tech225
sport302
tech265
sport041
tech251
tech243
politics076
sport189
entertainment360
entertainment370
entertainment286
sport433
business259
tech363
politics367
tech234
sport353
entertainment299
entertainment150
sport377
tech283
tech400
tech321
tech233
tech066
entertainment320
entertainment052
business028
business428
business411
entertainment131
business041
business487
entertainment128
sport020
politics302
tech082
business159
politics142
tech136
business355
politics101
politics145
business026
politics152
business109
sport346
entertainment323
business442
business390
entertainment376
sport344
politics348
entertainment025
politics022
entertainment007
sport143
entertainment316
tech331
politics275
sport391
tech192
tech176
politics393
entertainment273
entertainment262
politics346
business413
politics244
tech288
tech131
business098
entertainment296
politics241
sport383
business002
business139
tech262
sport019
tech252
politics347
politics178

politics139
politics365
sport142
sport264
business391
sport423
politics023
tech254
business264
business416
tech158
sport257
politics409
politics335
politics044
sport119
entertainment161
business060
tech394
entertainment003
sport427
sport347
sport368
business224
business278
politics084
entertainment233
sport464
politics298
sport402
tech168
business277
sport078
business432
sport117
entertainment165
tech201
tech358
sport291
politics201
politics019
politics239
business374
business401
politics072
tech055
sport245
entertainment279
sport299
business375
sport032
sport450
tech170
entertainment042
politics012
tech269
politics068
politics277
sport161
sport283
sport337
politics210
entertainment079
business221
entertainment041
tech157
politics408
tech094
tech097
business362
politics349
entertainment108
entertainment375
